In [ ]:
import sys
import os
import json
import jdata as jd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
from tabulate import tabulate
from glob import glob

from IPython.display import clear_output
from IPython.core.display import HTML
display(HTML("<style>.container {width:90% !important;}</style>"))
display(HTML("<style>pre { white-space: pre !important; }</style>"))



<div>
<img src="attachment:Screenshot%202022-08-01%20at%2013.49.35.png" width="500"/>
</div>

An LWE sample is a pair $(\textbf{a}, b: = \textbf{a}\cdot \textbf{s} + e)$, where $\textbf{a} \sim U(\mathbb{Z}_q^N)$
, $\textbf{s} \in \mathbb{Z}_q$ is the secret and $e \in \mathbb{Z}_q$ is the error from a small distribution, e.g. $\mathcal{N}(0,\sigma^2)$ with $\sigma=3.2$. We construct matrix $A_{N\times N}$ with $\textbf{a}$ as rows. \
Let $A' = \begin{bmatrix}
I_n & A \\
0 & qI_n 
\end{bmatrix}$. We run LLL on $A^{'}$, which finds an integer linear combinations of the rows that minimize the row norms. \
Denote the linear transformation (i.e., a change of basis, or a rotation) as matrix $R' = \begin{bmatrix} R_{2N\times N} & C_{2N\times N}  \end{bmatrix}$. 
We have $$ R'A' =  \begin{bmatrix} R_{2N\times N} & C_{2N\times N}  \end{bmatrix} \begin{bmatrix}I_n & A \\ 0 & qI_n \end{bmatrix} 
=\begin{bmatrix} R & RA+qC  \end{bmatrix} $$ \
where $qC$ effectively shifts the entries onto the interval $(-q/2,q/2)$. Hence, we get $RA$ which has small norm in the finite field. This is the purpose of the data preprocessing. 

In [ ]:
def getAb(N, Q, s):
    print(f'state {np.random.get_state}')
    print(f'N = {N}, Q = {Q}, s = {s}')
    A = np.random.randint(0,Q,size=(N,N))
    e = np.random.normal(0,3, N).astype(int)
    b = (A@s + e)%Q
    return A, b, e

def get2N(N, Q, A, b, e):
    AA = np.identity(2*N)
    AA[:N, N:] = A
    AA[N:, N:] *= Q

    bb = np.zeros(2*N)
    bb[:N] = b

    ee = np.zeros(2*N)
    ee[:N] = e
    return AA, bb, ee

N, Q = 5, 67
A, b, e = getAb(N, Q, np.random.binomial(1,  0.2, size = N))
AA, bb, ee = get2N(N, Q, A, b, e)
print(AA)



Above is an example of the $A'$ matrix. 
### Now we present a demo of how LLL runs. 

In [ ]:
class LLL:
    def __init__(self, Q = 251, delta = 0.75, right = True):
        self.delta = delta
        self.r = right
        self.Q = Q
    
    def proj_u(self, u, v):
        return self.proj_scale(u,v)*u
    
    def proj_scale(self, u, v):
        if np.dot(u, u) <1e-16:
            return 0
        return np.dot(u, v) / np.dot(u, u)
    
    def gram_schmidt(self, basis): 
        cb = basis.astype(float)
        orthobasis = np.zeros_like(cb)
        orthobasis[0] = cb[0]

        for i in range(1, len(basis)):
            orthobasis[i] = cb[i]
            for j in range(0, i):
                orthobasis[i] -= self.proj_u(orthobasis[j], cb[i])
        return orthobasis

    def size_reduction(self, basis, orthobasis, k):
        '''
        Performs length reduction on a basis.
        '''
        total_m = 0 
        for j in range(k-1, -1, -1):
            m = int(round(self.proj_scale(orthobasis[j], basis[k])))
            basis[k] -= m*basis[j] 
            self.R[k] -= m*self.R[j] 
            total_m += abs(m)

        if total_m > 0:
            orthobasis = self.gram_schmidt(basis)
    
        return basis, orthobasis

    def check_lovasz_condition_and_swap(self, basis, orthobasis, k):
        '''
        Checks the Lovasz condition for a basis. 
        Either swaps adjacent basis vectors and recomputes Gram-Schmidt or increments the working index.
        '''
        c = self.delta - self.proj_scale(orthobasis[k-1], basis[k])**2
        if orthobasis[k].dot(orthobasis[k]) >= c*orthobasis[k-1].dot(orthobasis[k-1]):
            k += 1
        else: 
            basis[[k, k-1]] = basis[[k-1, k]]
            self.R[[k, k-1]] = self.R[[k-1, k]]
            orthobasis = self.gram_schmidt(basis)
            k = max([k-1, 1])
        return basis, orthobasis, k

    def run(self, A, verbose = True):
        basis = A.copy()
        self.n = A.shape[0]
        self.m = self.n//2
        self.R = np.identity(self.n)
        k = 1
        orthobasis = self.gram_schmidt(basis)
        steps = 0
        
        self.init_log()
        
        self.maxk = k
        while k <= basis.shape[0] - 1:
            if steps % 5 == 0:
                self.log(basis, k)
            basis, orthobasis = self.size_reduction(basis, orthobasis, k)
            steps += 1

            basis, orthobasis, k = self.check_lovasz_condition_and_swap(basis, orthobasis, k)
            if verbose:
                print(tabulate(basis))
                clear_output(wait=True)
                time.sleep(0.2)
        return basis, self.R, self.stats
    
    def init_log(self):
        self.stats = dict()
        self.stats["r_norms"] = []
        self.stats["ra_norms"] = []
        self.stats["k"] = []
        self.stats["delta"] = self.delta
        self.stats["N"] = self.m
        self.stats['q25'] = []
        self.stats['q75'] = [] 
        self.stats['q37'] = []
        self.stats['q62'] = []
        
    def log(self, basis, k):
        if self.r: 
            R_rows = self.R[:,:self.m]
            B_rows = basis[:,self.m:]
        else:
            R_rows = self.R[:,self.m:]
            B_rows = basis[:,:self.m]
        self.stats["r_norms"].append(np.linalg.norm(R_rows, axis = 1).mean())
        b_rows = B_rows.copy()
        b_rows[b_rows > self.Q//2] -= self.Q
        self.stats["ra_norms"].append(np.linalg.norm(B_rows, axis = 1).mean())
        self.maxk = max(self.maxk, k)
        self.stats["k"].append(self.maxk)
        
        nonQId = []
        for row in B_rows:
            r = row%self.Q
            if sum(r) !=0:
                nonQId.extend(r)
        
        self.stats['q25'].append(np.quantile(nonQId, q = 0.25))   
        self.stats['q75'].append(np.quantile(nonQId, q = 0.75)) 
        self.stats['q37'].append(np.quantile(nonQId, q = 0.37))   
        self.stats['q62'].append(np.quantile(nonQId, q = 0.62)) 

# Show how LLL works on A' for a N = 20 example
lll = LLL(Q = 67, delta = 0.99)
aa_basis, R, stats = lll.run(AA, verbose = True)

Now, we try to get the corresponding $b$ of these $RA$, by applying the same linear transformation on $b$. That's simply $Rb$. 
Recall that in the original LWE samples, $b$ has error, which is also linearly combined by $R$: 

$$Rb = R(A\cdot s + e) = RAs + Re$$

Hence, our new samples $(\textbf{a}', b') = \left((RA)_i, (Rb)_i\right)$ has error $e_i' = (Rb)_i - (RA)_is = (Re)_i$. It has variance
$\mathbb{V}(Re) = R\mathbb{V}(e)R^\top = \sigma^2 RR^\top $. 

In the end, we get $2N$ samples with correlated errors, whose standard deviation is amplified by a factor of the $l_2$ norms of rows in $R$. 

Since acquiring $R$ is irrelevant to $b$, for the purpose of our experiments, for each $N,q$, we reuse the $A$ for different secrets in order to mitigate experiment cost. \
For more details on generating the datasets $(RA, Rb)$, refer to the code at src/generate/genSamples.py, the RA_Rb class. 

As LLL runs, the norms of $RA$ decreases (lower plot, green) and the norm of $R$ increases (lower plot, blue). 

In [ ]:
# Plot how the 25% quantile of entries in RA (A after LLL reduction) changes as LLL runs
x = range(len(stats['q25']))
plt.plot(x, stats['q25'])
plt.plot(x, Q - np.array(stats['q75']))
plt.plot(x, 0.5*(np.array(stats['q25']) + Q - np.array(stats['q75'])))

# Plot how the norm of R and RA changes as LLL runs. The norm of R is proportionate with the std of error. 
r, ra, ks = stats["r_norms"], stats["ra_norms"] , stats["k"] 
sz = len(r)
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(range(sz), ra, 'g-') 
ax2.plot(range(sz), r, 'b-')
# ax2.plot(range(sz), ks, 'r-', label = "max k")
ax2.set_ylabel("mean norm of rows of R",color='b')
ax1.set_ylabel( "mean norm of basis vectors", color='g')
plt.show()

However, this $R$ is too large for $Re$ to have reasonable standard deviation. 

So, in Picante, we introduce a penalty parameter $\omega$ in the $A'$ embedding. 

### Picante's data preprocessing
Let $A' = \begin{bmatrix}
\omega I_n & A \\
0 & qI_n 
\end{bmatrix}$. Then $R'A' = \begin{bmatrix} \omega R & RA+qC  \end{bmatrix}$, so a larger $\omega$ motivates a smaller $R$. 

And we switch to BKZ, a more advanced and more costly algorithm than LLL. Instead of projecting a vector on another, BKZ projects on the subspace spanned by a block of vectors. 

As shown below, a larger block size gives stronger reduction, and takes longer time. 

In [ ]:
def read_grids_data(path):
    '''
    Input: directory that stores json files
    Output: a list of dictionaries
    '''
    rows = []
    for filename in os.listdir(path):
        if not 'N' in filename:
            continue
        filepath = os.path.join(path, filename)
        stats = jd.load(filepath)
        for key in ['A', 'R', 'RA']:
            stats[key] = np.array(stats[key])

        N, Q = stats['N'], stats['Q']
        # Get rid of rows of all zeros
        nonzero_ind = [i for i in range(2*N) if np.any(stats['R'][i] != 0)]
        stats['RA'] = stats['RA'][nonzero_ind, :]
        stats['R'] = stats['R'][nonzero_ind, :]
        # Compute the norms
        RA = stats['RA'].copy() % Q
        RA[RA > Q//2] -= Q
        stats["ra_norms"] = np.linalg.norm(RA, axis = 1)
        stats["r_norms"] = np.linalg.norm(stats['R'], axis = 1)
        rows.append(stats)

    return rows

def plot_grid_search(df, fixed, variables):
    fig, axs = plt.subplots(2,3, figsize = (25, 15))
    for k1, k2, col in [(0,0,'time'), (0,1,'mean ||R||'), (0,2,'mean ||R|| / Q'), (1,0,'mean ||RA||'), (1,1,'mean ||RA|| / Q'), (1,2,'(std RA) / Q')]: 
        x, hue = variables['x'], variables['hue']
        ax = axs[k1, k2]
        mean_y_per_x = df.groupby(x).agg({col: 'mean' })
        sns.lineplot(data = mean_y_per_x, y = col, x = x, ax = ax)
        sns.scatterplot(data = df, x = x, y = col, hue = hue, ax = ax)
        ax.set_title(f"{col} per {x} and {hue} ")
    
    title_str = ', '.join([key + ' = ' + str(val) for key, val in fixed.items()])
    fig.suptitle(f"BKZ reduction results for {title_str}", fontsize=16,y=0.93)
    plt.show()
    plt.close()
    
grid_data = pd.DataFrame(read_grids_data('/path/to/bkz/grids/N100-p15.0-d0.99-full-log'))
grid_data = grid_data[grid_data['block_size']<25]

grid_data['log2 Q'] = grid_data['Q'].apply(np.log2).apply(np.ceil)
grid_data['mean ||R||'] = grid_data['r_norms'].apply(np.mean)
grid_data['mean ||R|| / Q'] = grid_data['r_norms'].apply(np.mean) / grid_data['Q']
grid_data['mean ||RA||'] = grid_data['ra_norms'].apply(np.mean)
grid_data['mean ||RA|| / Q'] = grid_data['ra_norms'].apply(np.mean) / grid_data['Q']
grid_data['(std RA) / Q'] = grid_data['RA'].apply(np.std) / grid_data['Q']

fixed = { 'N': 100, 'delta': 0.99, 'penalty': 15.0 }
variables = { 'x': 'block_size', 'hue': 'log2 Q' }
plot_grid_search(grid_data, fixed, variables)

def plot_RA(direc, Q):
    agg = set()
    for sample in read_grids_data(direc):
        if sample['Q'] != Q or sample['block_size'] > 30:
            continue
        params_tuple = (sample['N'], sample['Q'], sample['penalty'], sample['delta'], sample['block_size'])
        if params_tuple not in agg:
            agg.add(params_tuple)
            plt.hist(sample['ra_norms'], bins=20, label = f'bkz{sample["block_size"]}')
    plt.legend()
    
path = '/path/to/bkz/grids/N80-p15.0-d0.99-full-log'
plot_RA(path, 251)


### Visualize the distributions of the entries and the norms

In [ ]:
def plot_RA_distribution(direc, params):
    agg, ax2lims, ax4lims = {}, {}, {}
    for sample in read_grids_data(direc):
        if 'block_size' not in sample: #LLL
            sample['block_size'] = 2
        params_tuple = (sample['N'], sample['Q'], sample['penalty'], sample['delta'], sample['block_size'])
        if params_tuple not in agg:
            agg[params_tuple] = sample
            agg[params_tuple]['time'] = [sample['time']]
        else:
            agg[params_tuple]['RA'] = np.concatenate((agg[params_tuple]['RA'], sample['RA']))
            agg[params_tuple]['ra_norms'] = np.concatenate((agg[params_tuple]['ra_norms'], sample['ra_norms']))
            agg[params_tuple]['R'] = np.concatenate((agg[params_tuple]['R'], sample['R']))
            agg[params_tuple]['r_norms'] = np.concatenate((agg[params_tuple]['r_norms'], sample['r_norms']))
            agg[params_tuple]['time'].append(sample['time'])
        lim1, lim2 = (min(sample['ra_norms']), max(sample['ra_norms'])), (min(sample['r_norms']), max(sample['r_norms']))
        Q = sample['Q']
        if Q not in ax2lims:
            ax2lims[Q], ax4lims[Q] = lim1, lim2
        else:
            ax2lims[Q] = (min(ax2lims[Q][0], lim1[0]), max(ax2lims[Q][1], lim1[1]))
            ax4lims[Q] = (min(ax4lims[Q][0], lim2[0]), max(ax4lims[Q][1], lim2[1]))
    
    agg = sorted(list(agg.values()), key=lambda d: [d[p] for p in params]) 
    for stats in agg:
        stats['time'] = np.mean(stats['time'])
        f, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize = (30, 5))
        ax1.hist(stats['RA'].flatten() % stats['Q'], alpha = 0.5, bins=30);
        ax1.set_title(', '.join([f'{key} = {stats[key]}' for key in params]))
        ax2.hist(stats['ra_norms'], bins=20)
        ax2.set_title('||RA||')
        ax2.set_xlim(ax2lims[stats['Q']])
        e = np.int64(np.random.normal(0, 3, size = stats['N']).round())
        ax3.hist(stats['R']@e % stats['Q'], bins=20)
        ax3.set_title('error')
        ax4.hist(stats['r_norms'], bins=20)
        ax4.set_title('||R||')
        ax4.set_xlim(ax4lims[stats['Q']])
        plt.show()
        plt.close()

plot_RA_distribution('/path/to/bkz/grids/N100-p15.0-d0.99-full-log', ['Q', 'block_size', 'time'])

As shown above, lattice reduction on larger Q is more effective (more obvious change on the distribution), meaning that larger Q is easier. 
### The norm of $R$ is a proxy of how much the standard deviation of the error is amplified

In [ ]:
def plot_error_distribution(grid_search_data):
    N = grid_search_data[0]['N']
    s = np.zeros(shape=(N, 10), dtype=np.int64)
    for h in range(3, 13):
        idxs = np.random.choice(N, size = h, replace=False)
        s[idxs, h-3] = 1

    stats, ra_norms, r_norms, re_norms = [], {}, {}, {}
    for row in grid_search_data:
        e = np.int64(np.random.normal(0, 3, size = (N, 10)).round())
        Q = row['Q']
        b = (row['A']@s + e) % Q
        Rb = (row['R']@b) % Q
        Re = (row['RA']@s - Rb) % Q
        Re[Re > Q//2] -= Q
        stats.append({
            'Q': Q,
            # 'penalty': row['penalty'], 
            'Mean ||RA||': np.mean(row['ra_norms']), 
            'Mean ||R||': np.mean(row['r_norms']), 
            'std(Re) / std(e)': np.std(Re) / np.std(e), 
            # 'Mean Re': np.mean(Re),
            'std(Re)': np.std(Re),
            # 'std(e)': np.std(e),
        })
        if Q not in ra_norms:
            ra_norms[Q], r_norms[Q], re_norms[Q] = [],[],[]
        ra_norms[Q].extend(row['ra_norms'])
        r_norms[Q].extend(row['r_norms'])
        re_norms[Q].extend(np.linalg.norm(Re, axis=1))
    
    f, axs = plt.subplots(2,len(r_norms.keys()), figsize = (30, 12))
    for i,Q in enumerate(r_norms):
        axs[0,i].scatter(r_norms[Q], ra_norms[Q], 3)
        axs[0,i].set_title(f"Q={Q}")
        axs[0,i].set_xlabel('||R||')
        axs[0,i].set_ylabel('||RA||')
        axs[1,i].scatter(r_norms[Q], re_norms[Q], 3)
        axs[1,i].set_xlabel('||R||')
        axs[1,i].set_ylabel('||Re||')
    return pd.DataFrame(stats)
        
# Here we give an example to show that mean ||R|| approximates std(Re) / std(e)
# That means, the norm of R approximates the amplification of the std of error
path = '/path/to/bkz/grids/N80-p15.0-d0.99-full-log'
grid_80_rlwe = read_grids_data(path)
err_rlwe = plot_error_distribution(grid_80_rlwe)
err_rlwe.groupby('Q').mean()

### Statistics of RA and R are the metrics that quantify the preprocessing quality

In [ ]:
def get_data_stats(params, data, tiny=False):
    stats = {}
    Q = params['Q']
    tinyA = np.load('/path/to/tiny_A.npy')
    for U, R in data:
        if tiny:
            A = tinyA[U.flatten()]
        else:
            A = U
        nonzero_ind = [i for i in range(params['m']+params['N']) if np.any(R[i] != 0)]
        R = R[nonzero_ind, :]
        RA = (R@A) % Q
        if 'RA' not in stats:
            stats['RA'] = RA
            stats['R'] = R
            stats['nonzero'] = [len(nonzero_ind)]
        else:
            stats['RA'] = np.concatenate((stats['RA'], RA))
            stats['R'] = np.concatenate((stats['R'], R))
            stats['nonzero'].append(len(nonzero_ind))
    RA = stats['RA']
    RA[RA > Q//2] -= Q
    return stats, RA

def plot_hist(path):
    params, data = read_cluster_data(path)
    if len(data) == 0 or 'time' not in params:
        return
#         if params['block_size'] < 15 or params['pen'] != 15 or params['delta'] != 0.99: 
#             continue
    stats, RA = get_data_stats(params, data)
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize = (30, 5))

    ax1.hist(RA.flatten(), alpha = 0.5, bins=30);
    ax1.set_title(', '.join([f'{key} = {params[key]}' for key in ['Q', 'delta', 'pen', 'block_size']]))


    ax2.hist(np.linalg.norm(RA, axis=1), bins=20)
    print(RA.shape)
    ax2.set_title('||RA||')

    e = np.int64(np.random.normal(0, 3, size = params['m']).round())
    ax3.hist(np.array(stats['R'])@e % params['Q'], bins=20)
    ax3.set_title('error')

    ax4.hist(np.linalg.norm(stats['R'], axis=1), bins=20)
    ax4.set_title('||R||')

    plt.show()
    plt.close()

def save_grid_df(path, savepath=None):
    df = []
    for direc in os.listdir(path):
        p = os.path.join(path, direc)
        if len(p.split('stats'))==2:
            continue
        params, data = read_cluster_data(p, direc)
        if data == None:
            df.append(list(params.values()) + ['err'] * 3)
        elif len(data) == 0 or 'time' not in params:
            df.append(list(params.values()) + ['ip'] * 3)
        else:
            stats, RA = get_data_stats(params, data, False)
                
            df.append(list(params.values()) + [params['time']/np.mean(stats['nonzero']), np.sqrt(12)*np.std(RA)/params['Q'], np.mean(np.linalg.norm(stats['R'], axis=1))/params['Q']])
    param_cols = ['xp', 'N', 'Q', 'delta', 'pen', 'float type', 'block size', 'm', 'max loop']
    df = pd.DataFrame(df, columns = param_cols + ['time', 'time per line', 'std(RA)/std(rand)', '||R||/Q'])
    print(f'Saving {path}')
    if savepath:
        df.to_csv(os.path.join(savepath, 'stats.csv'))
    else:
        df.to_csv(os.path.join(path, 'stats.csv'))

n_grid = pd.read_csv('/path/to/stats.csv').sort_values('std(RA)/Q')
n_grid = n_grid[n_grid['strategy'] == False]
n_grid[n_grid['delta'] == 0.99]

In Picante, we select the parameters based on data frames like the one above. 

### Verde's data preprocessing
Let's look at the $A'$ matrix again: $\begin{bmatrix}
\omega I_n & A \\
0 & qI_n 
\end{bmatrix}$. \
The Q identity matrix are orthogonal, so if they are put first, there will be no projections on the first n rows, which should save time. \
Also, this way, when we reach the $A$, it would slide up and project with $qI_n$, so norm reduction is aided with mod Q
instead of just struggling to get short vectors using A without mod Q for a long time. \
In Verde, we rearrange the rows of $A'$: $\begin{bmatrix}
0 & qI_n \\
\omega I_n & A
\end{bmatrix}$. \
We use a even more efficient algorithm: BKZ 2.0 interleaved with Mark's new lattice reduction algo. Also, we introduce a large blocksize + early termination strategy. \
This is from the observation that most of the norm reduction is done in the first few loops of BKZ, as shown below. 

In [ ]:
n_grid = pd.read_csv('/path/to/n256_qfirst_loop/stats.csv').sort_values('max loop')
# n_grid = n_grid[n_grid['autoAbort'] == False]
# n_grid = n_grid[n_grid['strategy'] == False]
# n_grid = n_grid[n_grid['pen'] == 10]
n_grid = n_grid[n_grid['float type'] == 'dd']
n_grid = n_grid[n_grid['max loop'] < 9999]
display(n_grid)
plt.scatter([0]+n_grid['time'].tolist(), [1]+n_grid['std(RA)/std(rand)'].tolist(), s=10)
plt.xlabel('time') # minutes to generate one line
plt.ylabel('stddev/q')


### After Verde
Verde uses a lot of cpus. When processing 1 matrix, it only gets $2N$ samples at the end after hours or days of lattice reduction. \
So here's a new improvement: we start writing data once the norm reaches some threshold very close to the target, and let it run for a while (during which we keep writing data), 
so we harvest $\gg$2n samples from each matrix preprocessing. 

For more details, refer to the implementation at src/generate/genSamples.py, the BKZReducedLWE class. 

We also added support to RLWE. \
Let $R_q = \mathbb{Z}_q[x]/(x^n+1)$ be the set of polynomials whose degree is at most $n-1$ and coefficients are from $\mathbb{Z}_q$. \
An RLWE sample $(a(x), b(x): = a(x)\cdot s(x) + e(x))$ has coefficients $\textbf{a} \sim U(\mathbb{Z}_q^N)$
, $s(x) \in R_q$  is the secret and $e(x) \in R_q$ is the error. \
Equivalently, using vectors to represent the coefficients of the polynomials, we have $\textbf{b} = \textbf{A}^{circ}\cdot \textbf{s} + \textbf{e}$. 

Our data preprocessing preserve the circulant as follows:\
We take the first row of a list of circulant matrices to construct $A$, and reduce it (i.e., obtain the $R$ matrix). \
The norm is invariant on swapping or negating elements (which is what the circulant does to the rows), so apply the same linear transformation to the other rows of the matrices, would result in small norms as well. \
Since linear combination and taking the circulant are associative, we end up with circulant matrices from the reduced vectors. \
To get the corresponding $Rb$ of $(RA)_i^{circ}$, we just need to apply $R$ on the $b$ of other rows of $A^{circ}$. 


# uSVP attack
With $m$ LWE samples, we can construct matrix $A_{m\times N}$ by stacking the vectors as rows. Kannan's embedding is constructed as follows: $\begin{pmatrix}
I_n & A^T & 0\\
0 & qI_m & 0 \\
0 & b & 1
\end{pmatrix}$

An unusually short vector $\begin{pmatrix} s & -e & -1\end{pmatrix}$ is in the space spanned by these rows, since $\begin{pmatrix}
s & c & -1
\end{pmatrix}\begin{pmatrix}
I_n & A^T & 0\\
0 & qI_m & 0 \\
0 & b & 1
\end{pmatrix}  = \begin{pmatrix}
s & A \cdot s + qc - b & -1
\end{pmatrix}$.

In practice, the $I_n$ is multiplied by a parameter $\omega$ to balance $s$ and $-e$. \
For more details, refer to the implementation at src/generate/genSamples.py, the BenchmarkBKZ class. \
The parameters used by the paper "On the concrete security of lwe with small secret" can be reproduced below. 

In [ ]:
def compute_delta_d(n, logq, sigma):
    q = 2**logq
    w = np.round(sigma * np.sqrt(2))
    d = 2*n*np.log(q/w)/np.log(q/(np.sqrt(2*np.pi*np.e)*sigma))
    d_int = np.ceil(d)
    logdelta = np.log(q/(np.sqrt(2*np.pi*np.e)*sigma)) ** 2 / (4*n*np.log(q/w))
    return(d_int, np.round(np.e**logdelta, 4))

compute_delta_d(80, 9, 3.2), compute_delta_d(150, 15, 3.2), compute_delta_d(200, 19, 3.2)

Here's a new idea: if we just row reduce the right part of the matrix $\begin{pmatrix}
A^T \\ qI_m \\ b 
\end{pmatrix}$, we will still end up with an unusually short vector $e$ or $-e$. Then, we can subtract $e$ from $b$ to get the $A\cdot s$ without error. \
Now it is very vulnerable to algebraic attacks. Modular diagonalization can recover the secret as follows. 

In [ ]:
N, Q = 20, 251
Ab = np.zeros((N+2, N+1))
A = np.random.randint(0,Q,size=(N+2,N))
s = np.random.normal(0,3, size = N).astype(int)
Ab[:,:N] = A
Ab[:,-1] = A@s%Q

for i in range(N):
    k = 1
    while Ab[i,i] == 0:
        # it doesn't have an inverse. Swap with another row.
        Ab[i], Ab[i+k] = Ab[i+k], Ab[i]
        k += 1
    Ab[i] *= pow(int(Ab[i,i]), -1, Q) # this algo is in integer space. Multiply with the modular inverse
    Ab[i] %= Q
    for j in range(N):
        if i==j:
            continue
        Ab[j] -= Ab[j][i] * Ab[i] # gaussian elimination to diagonalize the matrix
        Ab[j] %= Q
s_solved = Ab[:N, -1]
s_solved[s_solved>Q//2] -=Q

np.all(s == s_solved)
